In [ ]:
import pandas as pd
import numpy as np
import math
from collections import defaultdict

In [10]:
file_path = 'sample_excel.xlsx'
df=pd.read_excel(file_path)


In [11]:
df.head()

,roll_no.,year,th
0,1,2024,57
1,2,2024,70
2,3,2024,59
3,4,2024,61
4,5,2024,65


In [13]:
# Assuming 'sample_excel.xlsx' has columns: 'year', 'roll_no.', 'th'
df_Endsem_res = pd.read_excel(file_path, sheet_name="Endsem_res_3y")

max_marks_dict = {
    2022: 80,
    2023: 80,
    2024: 80
}

# --- Calculations for Yearly Data ---

# Create the main summary DataFrame
summary = df_Endsem_res.groupby("year").agg(
    total_students=("roll_no.", "nunique"),
    th_total=("th", "sum")
)

summary["max_marks"] = summary.index.map(max_marks_dict)

# Calculate class average marks for each year
summary["class_avg_marks"] = summary["th_total"] / summary["total_students"]
summary["%_class_avg_marks"] =((summary["class_avg_marks"] / summary["max_marks"]) * 100)

# Calculate Students Who Achieved the Average
df_merged = pd.merge(df_Endsem_res, summary[['class_avg_marks']], on='year', how='left')
achieved_avg_count = df_merged[df_merged['th'] >= df_merged['class_avg_marks']].groupby('year').size()
summary['students_achieved_avg'] = achieved_avg_count
summary['%_students_achieved_avg'] = np.round(
    (summary['students_achieved_avg'] / summary['total_students']) * 100,
    2
)

# Continue with the other calculations
# Calculate Attainment Percentage
summary["attainment_%"] = np.ceil(
    (summary["th_total"] * 100) / (summary["total_students"] * summary["max_marks"])
)
# This column is for the scaled value, initialized with empty values
summary['attainment_scaled_3'] = np.nan


# --- NEW: Calculate and Add Overall Averages for ALL Columns ---

# 1. Calculate the mean of every column from the yearly data
overall_averages = summary.mean()

# 2. The scaled attainment is not a simple mean, so calculate it separately
#    based on the average of the 'attainment_%' column
avg_attainment_scaled = round((overall_averages['attainment_%'] * 3) / 100, 2)
# 3. Add the 'Overall Average' row using the calculated means
summary.loc['Overall Average'] = overall_averages

# 4. Place the specially calculated scaled value into the correct cell
summary.loc['Overall Average', 'attainment_scaled_3'] = avg_attainment_scaled


# --- Display and Export the Final DataFrame ---

print("--- Final Combined Summary DataFrame ---")
# Reordering columns for better readability
summary = summary[[
    'total_students', 
    'th_total', 
    'class_avg_marks', 
    '%_class_avg_marks', 
    'students_achieved_avg', 
    '%_students_achieved_avg', 
    'max_marks', 
    'attainment_%', 
    'attainment_scaled_3'
]]

# Rounding off numerical values for better presentation
summary = summary.round(2)

# Display the final table
summary



ValueError: Worksheet named 'Endsem_res_3y' not found

In [ ]:
#sheet 2 Goal set
stud_per_achieved_avg=math.ceil((summary['%_students_achieved_avg']['Overall Average']))
class_per_avg_marks=math.ceil((summary['%_class_avg_marks']['Overall Average']))
data = {
    "level":[1,2,3],
    "% of the student": [stud_per_achieved_avg-5,stud_per_achieved_avg,stud_per_achieved_avg+5],
    "% of marks": [class_per_avg_marks,class_per_avg_marks,class_per_avg_marks]
}

goalset = pd.DataFrame(data)
print(goalset)



   level  % of the student  % of marks
0      1                44          58
1      2                49          58
2      3                54          58


In [ ]:
import pandas as pd


def calculate_attainment_summary(file_path):

    # ============================
    # 1. LOAD MARKS SHEET
    # ============================
    df = pd.read_excel(file_path, sheet_name="IAT+tools")

    # Extract Max Marks row
    max_marks_row = df[df['Roll_no.'] == 'Max_Marks']
    if max_marks_row.empty:
        raise ValueError("'Max_Marks' row is missing in 'IAT+tools' sheet.")

    max_marks = max_marks_row.iloc[0].to_dict()
    max_marks.pop("Roll_no.")

    # Remove Max Marks Row
    df = df[df['Roll_no.'] != 'Max_Marks']
    total_students = len(df)

    # Convert marks safely
    for col in max_marks:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # ============================
    # 2. LOAD TOOL ASSIGNMENT
    # ============================
    tools_df = pd.read_excel(file_path, sheet_name="Tool Assignment")

    THRESHOLD = class_per_avg_marks
    output = []

    print("\n--- VALIDATION & DEBUG LOG ---")

    # ============================
    # 3. PROCESS EACH CO
    # ============================
    for index, row in tools_df.iterrows():

        co_id = row.get("CO-ID")
        co_statement = row.get("CO Statement")

        print(f"\nProcessing CO: {co_id}")

        # Read tools
        t1 = row.get("Tool1")
        t2 = row.get("Tool2")
        t3 = row.get("Tool3")

        # ============================
        # VALIDATION 1 — TOOL ORDER
        # ============================
        if pd.isna(t1) and (not pd.isna(t2) or not pd.isna(t3)):
            print("❌ ERROR: Tool1 is empty but Tool2/Tool3 is filled.")
            print("👉 Fix Excel: Always fill Tool1 first.")
            continue

        if pd.isna(t2) and not pd.isna(t3):
            print("❌ ERROR: Tool2 is empty but Tool3 is filled.")
            print("👉 Fix Excel: Do not skip Tool2.")
            continue

        # ============================
        # VALIDATION 2 — MINIMUM TOOLS
        # ============================
        tools = [t for t in [t1, t2, t3] if not pd.isna(t)]

        if len(tools) < 2:
            print("❌ ERROR: Less than 2 tools provided.")
            print("👉 At least TWO tools are required per CO.")
            continue

        # Extract CO number from ID
        try:
            co_number = int(str(co_id).split(".")[-1])
        except:
            print("❌ ERROR: CO-ID format invalid.")
            print("👉 Expected: format like '2343113.1' or 'CO1'")
            continue

        tool_results = []

        # ============================
        # 4. CALCULATION PER SLOT
        # ============================
        for slot_index, tool_name in enumerate(tools, start=1):

            column_name = f"CO{co_number}_tool_{slot_index}"

            print(f" Mapping: {tool_name} → {column_name}")

            # ============================
            # VALIDATION 3 — COLUMN EXISTS
            # ============================
            if column_name not in df.columns:
                print(f"❌ ERROR: Column '{column_name}' NOT FOUND in marks sheet.")
                print("👉 Check column names in 'IAT+tools' sheet.")
                continue

            max_mark = max_marks.get(column_name)

            # ============================
            # VALIDATION 4 — MAX MARKS VALID
            # ============================
            if pd.isna(max_mark) or max_mark == 0:
                print(f"❌ ERROR: Invalid max mark for {column_name}.")
                print("👉 Fix 'Max_Marks' row.")
                continue

            # ============================
            # CALCULATION
            # ============================
            percent_scores = (df[column_name] / max_mark) * 100
            achieved = (percent_scores >= THRESHOLD).sum()
            percent = (achieved / total_students) * 100

            tool_results.append({
                "tool": tool_name,
                "students": achieved,
                "percent": round(percent, 2)
            })

        # ============================
        # VALIDATION 5 — BEST TWO SAFETY
        # ============================
        if len(tool_results) < 2:
            print("❌ ERROR: Less than 2 valid tools after validation.")
            print("👉 Check marks sheet & mapping.")
            continue

        # ============================
        # 5. SELECT BEST TWO
        # ============================
        best_two = sorted(tool_results, key=lambda x: x['percent'], reverse=True)[:2]
        best_avg = round(
            (best_two[0]['percent'] + best_two[1]['percent']) / 2,
            2
        )

        # ============================
        # 6. OUTPUT FORMAT
        # ============================
        row_out = {"CO Statements": co_statement}

        for i, tr in enumerate(tool_results):
            row_out[f"Tool {i+1} Total"] = tr["students"]
            row_out[f"Tool {i+1} %"] = tr["percent"]

        row_out["% considering best 2 tools avg"] = best_avg
        output.append(row_out)

    # ============================
    # FINAL TABLE
    # ============================
    print("\n--- PROCESS COMPLETED ---")
    return pd.DataFrame(output)



if __name__ == "__main__":
    direct_attainment_df= calculate_attainment_summary(file_path)
    print(direct_attainment_df)



--- VALIDATION & DEBUG LOG ---

Processing CO: 2343113.1
 Mapping: IAT1 → CO1_tool_1
 Mapping: MCQ → CO1_tool_2
 Mapping: Assignment → CO1_tool_3

Processing CO: 2343113.2
 Mapping: IAT1 → CO2_tool_1
 Mapping: MCQ → CO2_tool_2
 Mapping: Mind Mapping → CO2_tool_3

Processing CO: 2343113.3
 Mapping: IAT1 → CO3_tool_1
 Mapping: MCQ → CO3_tool_2
 Mapping: Assignment → CO3_tool_3

Processing CO: 2343113.4
 Mapping: IAT2 → CO4_tool_1
 Mapping: MCQ → CO4_tool_2
 Mapping: Assignment → CO4_tool_3

Processing CO: 2343113.5
 Mapping: IAT2 → CO5_tool_1
 Mapping: MCQ → CO5_tool_2
 Mapping: Assignment → CO5_tool_3

Processing CO: 2343113.6
 Mapping: IAT2 → CO6_tool_1
 Mapping: MCQ → CO6_tool_2
 Mapping: Assignment → CO6_tool_3

--- PROCESS COMPLETED ---
                                       CO Statements  Tool 1 Total  Tool 1 %  \
0  Students will be able to discuss the need of a...            36     50.00   
1  Students will be able to analyze and design co...            59     81.94   
2  Studen

In [ ]:

# -----------------------------------
# LOAD DATA
# -----------------------------------
raw = pd.read_excel(file_path, sheet_name='ESE', header=None)

# -----------------------------------
# EXTRACT HEADERS
# -----------------------------------
questions = raw.iloc[0].ffill()
co_map = raw.iloc[1]
max_marks = raw.iloc[2]

# -----------------------------------
# FIND FIRST STUDENT ROW
# -----------------------------------
first_student_row = raw[
    raw.iloc[:, 0].apply(lambda x: str(x).strip().isdigit())
].index[0]

marks_df = raw.iloc[first_student_row:].reset_index(drop=True)

# First column = Roll No
roll_col = marks_df.iloc[:, 0]
marks_only = marks_df.iloc[:, 1:]

# Convert max marks to numeric
max_marks = pd.to_numeric(max_marks[1:], errors="coerce")

# Fix merged headers
questions = questions[1:].ffill()
co_map = co_map[1:]

# -----------------------------------
# PROCESS QUESTION-WISE
# -----------------------------------
results = []

for idx, col in enumerate(marks_only.columns):

    question = questions.iloc[idx]
    coa = co_map.iloc[idx]
    full_marks = max_marks.iloc[idx]

    # -----------------------------------
    # CLEANING LOGIC (IMPORTANT FIX)
    # -----------------------------------
    scores = marks_only[col]

    # Remove whitespace / empty cells
    scores = scores.replace(r'^\s*$', np.nan, regex=True)

    # Convert safely to numeric
    scores = pd.to_numeric(scores, errors="coerce")

    # Drop missing values
    scores = scores.dropna()

    # OPTIONAL: Remove zeros if zero = not attempted
    # scores = scores[scores > 0]

    if len(scores) == 0:
        continue

    # -----------------------------------
    # METRICS (FIXED)
    # -----------------------------------
    attempted = scores.count()

    class_avg = scores.mean()
    class_avg_percent = (class_avg / full_marks) * 100

    above_threshold = ((scores / full_marks) * 100 >= class_per_avg_marks).sum()
    percent_above = (above_threshold / attempted) * 100

    # -----------------------------------
    # STORE
    # -----------------------------------
    results.append({
        "Question": question,
        "CO": coa,
        "Max Marks": full_marks,
        "No. of Students Attempted": attempted,
        "No. >= Threshold": above_threshold,
        "% >= Threshold": round(percent_above, 2),
        "% Class Avg Marks": round(class_avg_percent, 2)
    })

# -----------------------------------
# FINAL DATAFRAME
# -----------------------------------
final_df = pd.DataFrame(results)

# SHOW RESULT
final_df


NameError: name 'file_path' is not defined

In [ ]:
CES_df = pd.read_excel("sample_excel.xlsx",sheet_name="CES")

# Step 1: Number of students
student_num = CES_df["Sr. No"].nunique()
print("Number of students:", student_num)

# Step 2: Create dictionary to hold results
results = {}

# Step 3: Loop through CO1 to CO6
for co in [c for c in CES_df.columns if c.startswith("CO")]:
    counts = CES_df[co].value_counts()

    # Raw numbers
    num_3 = counts.get(3, 0)
    num_2 = counts.get(2, 0)
    num_1 = counts.get(1, 0)

    # Percentages
    perc_3 = (num_3 * 100) / student_num
    perc_2 = (num_2 * 100) / student_num
    perc_1 = (num_1 * 100) / student_num

    results[co] = {
        "3's (num)": num_3, "3's (%)": perc_3,
        "2's (num)": num_2, "2's (%)": perc_2,
        "1's (num)": num_1, "1's (%)": perc_1,
        "Overall Average %": np.round((perc_3 * 3 + perc_2 * 2 + perc_1 * 1) / 3,2)
    }


# Step 4: Convert to DataFrame for better display
indirect_attainment_df = pd.DataFrame(results).T

print(indirect_attainment_df)


Number of students: 72
     3's (num)    3's (%)  2's (num)    2's (%)  1's (num)   1's (%)  \
CO1       44.0  61.111111       28.0  38.888889        0.0  0.000000   
CO2       37.0  51.388889       35.0  48.611111        0.0  0.000000   
CO3       38.0  52.777778       34.0  47.222222        0.0  0.000000   
CO4       40.0  55.555556       31.0  43.055556        1.0  1.388889   
CO5       38.0  52.777778       34.0  47.222222        0.0  0.000000   
CO6       39.0  54.166667       32.0  44.444444        1.0  1.388889   

     Overall Average %  
CO1              87.04  
CO2              83.80  
CO3              84.26  
CO4              84.72  
CO5              84.26  
CO6              84.26  


In [ ]:
# ================================
# NBA STYLE MULTI-ROW REPORT
# ================================

# 1) DEFINE MULTI-LEVEL HEADER
multi_columns = pd.MultiIndex.from_tuples([

    # ------- DIRECT ASSESSMENT (90%) -------
    ("Direct Assessment (90%)", "Internal Assessment", "Tool 1"),
    ("Direct Assessment (90%)", "Internal Assessment", "Tool 2"),
    ("Direct Assessment (90%)", "Internal Assessment", "Tool 3"),

    ("Direct Assessment (90%)", "Internal Attainment %", ""),
    ("Direct Assessment (90%)", "Internal Attainment Level", ""),

    ("Direct Assessment (90%)", "End Sem Exam Attainment %", ""),
    ("Direct Assessment (90%)", "End Sem Exam Attainment Level", ""),

    ("Direct Assessment (90%)", "Direct Attainment Level", ""),

    # ------- INDIRECT ASSESSMENT (10%) -------
    ("Indirect Assessment (10%)", "Course Exit Survey", ""),
    ("Indirect Assessment (10%)", "Indirect Attainment Level", ""),

    # ------- FINAL / TARGET / RESULT -------
    ("Final", "Final Attainment Level (Direct 90% + Indirect 10%)", ""),
    ("Target", "CO Attainment Target", ""),
    ("Result", "CO Attainment", "")
])

# 2) CREATE EMPTY REPORT WITH MULTI-HEADERS
final_report_df = pd.DataFrame(index=final_CO_attainment_df.index, columns=multi_columns)

# 3) MAP VALUES INTO NBA FORMAT

# --- Internal Assessment Tools ---
final_report_df[("Direct Assessment (90%)", "Internal Assessment", "Tool 1")] = CO_attain_df["Tool 1"]
final_report_df[("Direct Assessment (90%)", "Internal Assessment", "Tool 2")] = CO_attain_df["Tool 2"]
final_report_df[("Direct Assessment (90%)", "Internal Assessment", "Tool 3")] = CO_attain_df["Tool 3"]

# --- Internal Attainment ---
final_report_df[("Direct Assessment (90%)", "Internal Attainment %", "")] = final_CO_attainment_df["Internal Attainment %"]
final_report_df[("Direct Assessment (90%)", "Internal Attainment Level", "")] = final_CO_attainment_df["Internal Attainment Level"]

# --- End Sem Attainment ---
final_report_df[("Direct Assessment (90%)", "End Sem Exam Attainment %", "")] = final_CO_attainment_df["End Sem Exam Attainment %"]
final_report_df[("Direct Assessment (90%)", "End Sem Exam Attainment Level", "")] = final_CO_attainment_df["End Sem Exam Attainment Level"]

# --- Direct Level ---
final_report_df[("Direct Assessment (90%)", "Direct Attainment Level", "")] = final_CO_attainment_df["Direct Attainment Level"]

# --- Indirect ---
final_report_df[("Indirect Assessment (10%)", "Course Exit Survey", "")] = final_CO_attainment_df["Course Exit Survey"]
final_report_df[("Indirect Assessment (10%)", "Indirect Attainment Level", "")] = final_CO_attainment_df["Indirect Attainment Level"]

# --- Final / Target / Result ---
final_report_df[("Final", "Final Attainment Level (Direct 90% + Indirect 10%)", "")] = final_CO_attainment_df["Final Attainment Level"]
final_report_df[("Target", "CO Attainment Target", "")] = TARGET_ATTAINMENT_LEVEL
final_report_df[("Result", "CO Attainment", "")] = final_CO_attainment_df["CO Attainment"]

# 4) ROUND
final_report_df = final_report_df.round(2)

# 5) SHOW IN NOTEBOOK
final_report_df


NameError: name 'final_CO_attainment_df' is not defined